In [1]:
import numpy as np
from sklearn.naive_bayes import GaussianNB

from helper import extract_feature, save_model
from preprocess import get_training_data


def _get_training_feature():
    training_images, training_labels = get_training_data()
    img_features, codebook = extract_feature(training_images)
    return img_features, codebook, training_labels




def train_nb(model_directory: str = "models"):
    img_features, codebook, training_labels = _get_training_feature()

    # train a Naive Bayes classifier
    model = GaussianNB()
    estimator = model.fit(img_features, np.array(training_labels))

    # For testing and reuse
    save_model(model_directory + "/nb_estimator.pkl", estimator)
    save_model(model_directory + "/nb_codebook.pkl", codebook)


train_nb()

ModuleNotFoundError: No module named 'cv2'

In [25]:
import cv2
import numpy as np
from scipy.cluster.vq import vq

from config import config
from helper import load_model
from preprocess import get_test_data


def evaluate_model(estimator_path: str, codebook_path: str):
    estimator = load_model(estimator_path)
    codebook = load_model(codebook_path)

    orb = cv2.ORB_create()
    test_images, test_labels = get_test_data()
    n = len(test_images)

    test_descriptors = []
    # extract features
    for image_path in test_images:
        img = cv2.imread(image_path)
        features = orb.detect(img, None)
        _, img_descriptor = orb.compute(img, features)
        test_descriptors.append((image_path, img_descriptor))

    img_features = np.zeros((n, config.CLUSTER_SIZE), "float32")
    # create histogram of test images
    for i in range(n):
        words, distance = vq(test_descriptors[i][1], codebook)
        for word in words:
            img_features[i][word] += 1

    predictions = estimator.predict(img_features)
    correct = 0
    for i in range(n):
        if predictions[i] == test_labels[i]:
            correct += 1

    accuracy = correct / n
    return round(accuracy, 2)


if __name__ == "__main__":

    nb_accuracy = evaluate_model("models/nb_estimator.pkl", "models/nb_codebook.pkl")
    print(f"Naive Bayes Classification Accuracy: {nb_accuracy * 100}%")

Naive Bayes Classification Accuracy: 44.0%
